# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import json
import csv
import numpy as np
import pandas as pd
import joblib

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.core import Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core.conda_dependencies import CondaDependencies

In [ ]:
# Create the CPU computer cluster
amlcompute_cluster_name = "capstone"

# Verify if cluster does not exist otherwise use the existing one
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_V2',
                                                           vm_priority = 'lowpriority', 
                                                           max_nodes=5,
                                                           min_nodes=0,
                                                           idle_seconds_before_scaledown=600)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-courtlin",subscription_id="bd6c48f0-b2f5-4fd8-b4de-3351b13cbee8", resource_group="udacity-nano-degree")
exp = Experiment(workspace=ws, name="hyperdrive-ml")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
# ws = Workspace.from_config()
# experiment_name = 'hyperdrive-ml'

# experiment=Experiment(ws, experiment_name)

In [ ]:
#Access the external data needed for the project that has been downloaded from Kaggle and uploaded to the Azure workspace
#This dataset, title Stroke Prediction Dataset, was downloaded from Kaggle for educational use. URL: https://www.kaggle.com/fedesoriano/stroke-prediction-dataset?select=healthcare-dataset-stroke-data.csv
#The author of the dataset is fedesoriano

# Load the registered dataset from workspace
dataset = Dataset.get_by_name(ws, name='healthcare-dataset-stroke-data')

# from azureml.data.dataset_factory import TabularDatasetFactory
# from azureml.core.dataset import Dataset

# url_paths = [
#     "https://www.kaggle.com/fedesoriano/stroke-prediction-dataset/download"
#     ]

# # dataset = TabularDatasetFactory.from_delimited_files(path=url_paths)
# dataset = Dataset.Tabular.from_delimited_files(path=url_paths)
# type(dataset)

# Convert the dataset to dataframe
df = dataset.to_pandas_dataframe()
df.info()

In [ ]:
df.describe()

In [ ]:
df.head(10)

In [ ]:
# Create the virtual environment
environment = Environment(name="azure-env")
conda_env = CondaDependencies()

# Needed packages and scripts
conda_env.add_conda_package("pandas")
conda_env.add_conda_package("numpy")
conda_env.add_conda_package("scikit-learn")
conda_env.add_pip_package("azureml-defaults")

# Adding dependencies to the created environment
environment.python.conda_dependencies=conda_env

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
# Define an early termination policy
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2)

#TODO: Create the different params that you will be using during training
# Create different params that will be used during training
param_sampling = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1.0,10.0,50.0),
        '--max_iter': choice(10,25)
    }
)

#TODO: Create your estimator and hyperdrive config
# Create estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory = ".",
                            script='train.py',
                            arguments=['--data', dataset.id],
                            compute_target=amlcompute_cluster_name,
                            environment=environment)

hyperdrive_run_config = HyperDriveConfig(run_config=estimator,
                                     hyperparameter_sampling=param_sampling, 
                                     policy=early_termination_policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=24,
                                     max_concurrent_runs=4)

In [ ]:
#TODO: Submit your experiment
run = experiment.submit(hyperdrive_run_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(run).show()
run.wait_for_completion(show_output=True)

In [ ]:
run

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
# Retrieve and save best model
best_run = run.get_best_run_by_primary_metric()
best_run.get_file_names()

In [ ]:
#TODO: Save the best model


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Register the best model with Azure
model = best_model.register_model(model_name='HYPERdrive-best-stroke-model',model_path='outputs/model.pkl')

# Get the AutoML environment details with its dependencies
environment = Environment.get(ws, "AzureML-HyperDrive")


In [ ]:
print(environment)

In [ ]:
inference_config = InferenceConfig(entry_script='scoring.py',
                                   environment=environment)
service_name = 'HYPERdrive-best-stroke-deployed'
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, enable_app_insights=True, auth_enabled=True)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config,                       
                       overwrite=True
                      )
service.wait_for_deployment(show_output=True)

scoring_uri = service.scoring_uri
print(scoring_uri)

In [ ]:
scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')
print(f'swagger URI: \n{service.swagger_uri}\n')



TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
data = {
  "data": [
    {
      "gender": "Male",
      "age": 67,
      "hypertension": "False",
      "heart_disease": "True",
      "ever_married": "True",
      "work_type": "Private",
      "Residence_type": "Urban",
      "avg_glucose_level": 228.69,
      "bmi": 36.6,
      "smoking_status": "formerly smoked"
    },
    {
      "gender": "Female",
      "age": 50,
      "hypertension": "False",
      "heart_disease": "False",
      "ever_married": "False",
      "work_type": "0.0",
      "Residence_type": "0.0",
      "avg_glucose_level": 0,
      "bmi": 0,
      "smoking_status": "0"
    }
  ]
}

# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {"Content-Type": "application/json"}
# If authentication is enabled, set the authorization header
headers["Authorization"] = f"Bearer {key}"

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
#Print the logs from the service
logs = service.get_logs()
logs

In [ ]:
# Delete the deployed model
service.delete()

#Delete the CPU compute target
# compute_target.delete()